In [2]:
import googlemaps
import pandas as pd
import time

In [ ]:
# Your Google API key
API_KEY = "YourKey"

# Initialize the Google Maps client
gmaps = googlemaps.Client(key=API_KEY)

# Get coordinates of Gravatá
localizacao = "Gravatá, Pernambuco, Brazil"
geocode_result = gmaps.geocode(localizacao)

# Check if the geocoding was successful
if not geocode_result:
    raise Exception("Could not get the coordinates of Gravatá.")

# Extract latitude and longitude
lat_lng = geocode_result[0]["geometry"]["location"]
latitude = lat_lng["lat"]
longitude = lat_lng["lng"]

# Search parameters
tipo = "restaurant"
radius = 5000  # Search radius in meters

# List to store the restaurants
restaurantes = []
next_page_token = None

while True:
    # Make the initial request or fetch the next page
    if next_page_token:
        resultados = gmaps.places_nearby(location=(latitude, longitude), radius=radius, type=tipo, page_token=next_page_token)
    else:
        resultados = gmaps.places_nearby(location=(latitude, longitude), radius=radius, type=tipo)
    
    # Add the found restaurants
    restaurantes.extend(resultados["results"])

    # Check if there are more pages of results
    next_page_token = resultados.get("next_page_token")
    
    if not next_page_token:
        break  # If there are no more pages, exit the loop

    # Google may take a few seconds to release the next page
    time.sleep(2)

# Create the table with restaurant details
dados = []
for lugar in restaurantes:
    detalhes = gmaps.place(place_id=lugar["place_id"])["result"]
    
    nome = detalhes.get("name", "N/A")
    endereco = detalhes.get("formatted_address", "N/A")
    telefone = detalhes.get("formatted_phone_number", "N/A")
    site = detalhes.get("website", "N/A")
    horario = detalhes.get("opening_hours", {}).get("weekday_text", "N/A")
    email = detalhes.get("email", "N/A")  # The API usually does not provide email

    dados.append([nome, endereco, telefone, site, horario, email])
    
    # Pause to avoid API blocking
    time.sleep(1)

# Create DataFrame and save to Excel
df = pd.DataFrame(dados, columns=["Name", "Address", "Phone", "Website", "Hours", "Email"])
df.to_excel("restaurantes_gravata.xlsx", index=False)

print("Excel file generated successfully!")


Excel file generated successfully!
